In [1]:
import chess
import math

## Evaluation of the given state
1. evaluate piece values
2. evaluate pawn position state
- 2.1. evaluate blocked pawns
- 2.2 evaluate doubled pawns
- 2.3. evaluate isolated pawns
3. evaluate mobility -open-
4. evaluate stalemate and 75 moves rule -open-

In [7]:
# Maximum number of moves is 5898 (due to the 50 Moves Rule - that makes this number big enough)
WIN_VALUE = 100000

def static_eval(board):
    if board.result() == "1-0":
        return WIN_VALUE - board.fullmove_number
    elif board.result() == "0-1":
        return -WIN_VALUE + board.fullmove_number
    elif board.result() == "1/2-1/2" or board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
        return 0
    
    value = 0
    blocked_pawns_white = 0
    doubled_pawns_white = 0
    isolated_pawns_white = 0
    blocked_pawns_black = 0
    doubled_pawns_black = 0
    isolated_pawns_black = 0
    
    
    # 1. evaluate piece values -------------------------------------------
    # Types:
    """{chess.PAWN, chess.KNIGHT, chess.BISHOP,
             chess.ROOK, chess.QUEEN, chess.KING}"""
    piece_values = [1, 
                    3, 
                    3, 
                    5,
                    9]
    for i, piece_value in enumerate(piece_values): # https://www.chessprogramming.org/Simplified_Evaluation_Function
        # BAUERN
        if i == 0:
            legal_moves = set(board.legal_moves)
            file_white = [0, 0, 0, 0, 0, 0, 0, 0]
            file_black = [0, 0, 0, 0, 0, 0, 0, 0]
            
            for square in board.pieces(1, chess.WHITE):
                pawn_blocked = True
                move_one = chess.Move(square, square + 8)
                move_two = chess.Move(square, square + 16)
                move_three = chess.Move(square, square + 7)
                move_four = chess.Move(square, square + 9)
                if move_one in legal_moves or move_two in legal_moves or move_three in legal_moves or move_four in legal_moves:
                    pawn_blocked = False
                if pawn_blocked:
                    blocked_pawns_white += 1
                if file_white[chess.square_file(square)] == 1:
                    doubled_pawns_white += 1
                else:
                    file_white[chess.square_file(square)] = 1
            if file_white[0] == 1 and file_white[1] == 0:
                isolated_pawns_white += 1
            if file_white[7] == 1 and file_white[6] == 0:
                isolated_pawns_white += 1
            for j in range(1, 7):
                if file_white[i] == 1 and file_white[i+1] == 0 and file_white[i-1] == 0:
                    isolated_pawns_white += 1
                    
            for square in board.pieces(1, chess.BLACK):
                pawn_blocked = True
                move_one = chess.Move(square, square - 8)
                move_two = chess.Move(square, square - 16)
                move_three = chess.Move(square, square - 7)
                move_four = chess.Move(square, square - 9)
                if move_one in legal_moves or move_two in legal_moves or move_three in legal_moves or move_four in legal_moves:
                    pawn_blocked = False
                if pawn_blocked:
                    blocked_pawns_black += 1
                if file_black[chess.square_file(square)] == 1:
                    doubled_pawns_black += 1
                else:
                    file_black[chess.square_file(square)] = 1
            if file_black[0] == 1 and file_black[1] == 0:
                isolated_pawns_black += 1
            if file_black[7] == 1 and file_black[6] == 0:
                isolated_pawns_black += 1
            for j in range(1, 7):
                if file_black[i] == 1 and file_black[i+1] == 0 and file_black[i-1] == 0:
                    isolated_pawns_black += 1
        # SPRINGER AM RAND BEDEUTET KUMMER UND SCHAND' 
        if i == 1:
            for square in board.pieces(2, chess.WHITE):
                if chess.square_file(square) == 0 or chess.square_file(square) == 7:
                    value -= 0.3
            for square in board.pieces(2, chess.BLACK):
                if chess.square_file(square) == 0 or chess.square_file(square) == 7:
                    value += 0.3
                    
        # LÄUFER AM RAND BEDEUTET KUMMER UND UMMER (UND AUCH 'N BISSCHEN SCHAND')
        if i == 2:
            for square in board.pieces(3, chess.WHITE):
                if chess.square_file(square) == 0 or chess.square_file(square) == 7:
                    value -= 0.1
            for square in board.pieces(3, chess.BLACK):
                if chess.square_file(square) == 0 or chess.square_file(square) == 7:
                    value += 0.1
                                
        value += len(board.pieces(i+1, chess.WHITE))*piece_value
        value -= len(board.pieces(i+1, chess.BLACK))*piece_value
    
    # 2.1 Blocked Pawns
    if not board.is_check():
        value -= 0.1 * (blocked_pawns_white - blocked_pawns_black)
    
    # 2.2 Doubled Pawns
    value -= 0.5 * (doubled_pawns_white - doubled_pawns_black)
    
    # 2.3 Isolated Pawns
    value -= 0.5 * (isolated_pawns_white - isolated_pawns_black)
    
    return value

## Calculate how many doubled pawns exist for a given file_dict

In [3]:
def count_doubled_pawns(file_dict):
    return len([file_count for file_count in file_dict if file_dict[file_count]>1])

## Calculate isolated pawns

In [4]:
def has_adjacent_pawn(file_dict,file_count):
    if file_count == 1:
        return file_dict[2]
    elif file_count == 8:
        return file_dict[7]
    else:
        return file_dict[file_count+1] or file_dict[file_count-1]

In [5]:
def count_isolated_pawns(file_dict):
    return len([file_count for file_count in file_dict if not has_adjacent_pawn(file_dict,file_count)])

## Analyze Pawns for a given color:
- Returns: tuple:
    - blocked_pawns 
    - doubled_pawns
    - isolated_pawns
    
Steps:
1. Build dict: {file_number:pawn_count} --> {1:2, 2:0, 3:1} while counting blocked pawns
2. calculate doubled pawns
3. calculate isolated pawns
4. return (blocked_pawns, doubled_pawns, isolated_pawns)

In [6]:
def analyze_pawns(board,color):
    file_dict = {}
    legal_moves = set(board.legal_moves)
    blocked_pawns = 0
    
    for file_count, file in enumerate(chess.FILE_NAMES,start=1):
        file_dict[file_count] = 0
        for rank in chess.RANK_NAMES:
            square = chess.parse_square(file + rank)
            if board.piece_type_at(square) == chess.PAWN and board.color_at(square) == color:
                file_dict[file_count] += 1
                
                pawn_blocked = True
                for move in legal_moves:
                    if move.from_square == square:
                        pawn_blocked = False
                        legal_moves.remove(move)
                        break
                if pawn_blocked:
                    blocked_pawns+=1
    
    
    return (blocked_pawns,count_doubled_pawns(file_dict),count_isolated_pawns(file_dict))

## Main minimax function

In [7]:
def minimax(board, depth, maximizing_player, moves):
    if depth == 0 or (not board.legal_moves):
        return (static_eval(board), moves)
        

    if maximizing_player:
        max_val = -math.inf
        max_move = None
        for move in board.legal_moves:
            board.push(move)
            value, _ = minimax(board, depth - 1, False, moves + [move])
            board.pop()
            max_val = max(max_val, value)
            if max_val == value:
                max_move = move
        assert board.legal_moves
        assert not max_move == None
        return max_val, moves + [max_move]
    else:
        min_val = +math.inf
        min_move = None
        for move in board.legal_moves:
            board.push(move)
            value, _ = minimax(board, depth - 1, True, moves + [move])

            board.pop()
            min_val = min(min_val, value)
            if min_val == value:
                min_move = move
        assert board.legal_moves
        assert not min_move == None
        return min_val, moves + [min_move]

In [8]:
def minimax_for_color(board, color, depth):
    return minimax(board, depth, color, [])